<div class="alert alert-block alert-warning" align=center><b>
    LION KING MOVIE : Reviews Sentiment Analysis</b>
</div>

## Importing Required Packages

In [1]:
# Usual packages 
import os
import json
import requests
import datetime
import time
import joblib
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)
from pandas.io.json import json_normalize 
from itertools import chain
from sklearn.base import BaseEstimator, TransformerMixin
from itertools import compress
from contractions import CONTRACTION_MAP

# text related packages
import re
import unicodedata
import emoji
import spacy
# Load the language model
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS

import gensim
import textblob
from textblob import TextBlob
from nltk.tokenize import word_tokenize


## modelling related packages
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import ADASYN

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import StandardScaler

## Scrapping Data from Web site: www.RottenTomatoes.com

In [2]:
def web_scrapping(collect=False,no_pages=None,collection_dir=None):
    
    if(collect):
        
        # Settings required for scrapping
        headers = {
         'Referer': 'https://www.rottentomatoes.com/m/the_lion_king_2019/reviews?type=user',\
         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, likeGecko) Chrome/74.0.3729.108 Safari/537.36',\
         'X-Requested-With': 'XMLHttpRequest',\
        }


        url = 'https://www.rottentomatoes.com/napi/movie/9057c2cf-7cab-317f-876f-e50b245ca76e/reviews/user'


        payload = {
            'direction': 'next',
            'endCursor': '',
            'startCursor': ''
        }


        ### Scrap page by page as allowed

        parent_path =os.getcwd()
        ## Storing all the json data in movie_rdata dir
        reviews_path = parent_path +"\\"+collection_dir
        os.chdir(reviews_path)


        s = requests.Session()


        i=0
        while (i < no_pages):
            time.sleep(5)
            data=''
            r=''

            #print(payload,"i=",i)

            r = s.get(url, headers=headers, params=payload) # GET Call
            data = r.json()
            #print(data)

            if(data['pageInfo']['hasNextPage']):
                next_endCursor=data['pageInfo']['endCursor']

            payload = {
                'direction': 'next',
                'endCursor': next_endCursor,
                'startCursor': ''
            }

            filename="page"+str(i)+".json"
            with open(filename, 'w') as json_file:
                json.dump(data, json_file)
            i=i+1

            json_file.close()



In [3]:
scrap_dir = "movie_rdata"
total_pages = 300 #default

scrap_choice=input("Would you like to continue with fresh web-scrapping [yes:no]")
if(scrap_choice=='yes'):
    web_scrapping(collect=True,no_pages=total_pages,collection_dir=scrap_dir)
else:
    scrap_dir=input("Please enter the repository[Dir] from which to process the collected webscrapped pages")


Would you like to continue with fresh web-scrapping [yes:no]no
Please enter the repository[Dir] from which to process the collected webscrapped pages../movie_rdata


## Process the collected data and populate the input DF

In [4]:
# Ask User for Movie Collection and the Directory to store

parent_path =os.getcwd()
reviews_path = parent_path +"\\"+scrap_dir
os.chdir(reviews_path)

# Process data from the collected Dir and make a df
print(reviews_path)

total_files_processed=0

for review_file in os.listdir(reviews_path):
    if review_file.endswith(".json"):
        with open(review_file) as infile:
            #print(review_file)
            jdata=json.load(infile)
            if(total_files_processed>0):
                movie_df=movie_df.append(json_normalize(jdata['reviews']))
                #print("coming here",total_files_processed)
            else:
                movie_df=json_normalize(jdata['reviews'])
                #print("first time",total_files_processed)
          
            total_files_processed=total_files_processed+1

            
print("Total files processed=",total_files_processed)
os.chdir(parent_path)

C:\Users\trex\Downloads\ML Comp Practice\PHD\final_submisstion\../movie_rdata
Total files processed= 300


In [5]:
# resetting index as each page indexes page as 0-9 from json collection method
movie_df=movie_df.reset_index(drop=True)

In [6]:
movie_df.head(12)

,createDate,displayImageUrl,displayName,hasProfanity,hasSpoilers,isSuperReviewer,isVerified,rating,review,score,timeFromCreation,updateDate,user.accountLink,user.displayName,user.realm,user.userId
0,2019-08-18T08:54:30.664Z,None,Joanne H,False,False,False,True,STAR_5,I liked most that the animation made the anima...,5.0,1h ago,2019-08-18T08:54:30.890Z,None,Joanne H,Fandango,2c73ed20-5b9f-41b3-a4fd-8dd3ff8bb20a
1,2019-08-18T08:03:49.380Z,https://graph.facebook.com/v3.3/594379764/picture,Frankie C,False,False,False,False,STAR_5,Amazing! So realistic and incredible music,5.0,2h ago,2019-08-18T08:03:49.380Z,/user/id/871398953,Frankie C,RT,871398953
2,2019-08-18T07:13:32.422Z,None,jaycee,False,False,False,False,STAR_5,Classic. Good remake. Loved it. Glover was out...,5.0,3h ago,2019-08-18T07:13:32.422Z,None,jaycee,Fandango,DD2453B0-37CE-4B47-A099-D15378FC310E
3,2019-08-18T07:06:08.698Z,https://graph.facebook.com/v3.3/10000047937306...,Peter A,False,False,False,False,STAR_2,Nice animation/CGI but completely lacking the ...,2.0,3h ago,2019-08-18T07:06:46.610Z,/user/id/906750266,Peter A,RT,906750266
4,2019-08-18T06:38:46.892Z,None,Kevin M,False,False,False,True,STAR_4,Good but go mainly for the sentimental value,4.0,3h ago,2019-08-18T06:38:46.892Z,None,Kevin M,Fandango,21FE7D93-351C-43B5-833D-B1DB11F3FD6A
5,2019-08-18T05:34:19.360Z,None,Angelwolf,False,False,False,True,STAR_4,The animation was really good,4.0,4h ago,2019-08-18T05:34:19.360Z,None,Angelwolf,Fandango,E62CDAD2-329A-4924-8455-35249C3BCC4D
6,2019-08-18T05:22:03.783Z,None,sheril,False,False,False,True,STAR_5,i thought it was awesone. i really like the or...,5.0,5h ago,2019-08-18T05:22:03.783Z,None,sheril,Fandango,8229f609-93a7-4c8e-8ec7-37b18fa65679
7,2019-08-18T05:14:42.958Z,None,Maggie L,False,False,False,True,STAR_3,I think that they should’ve brought back more ...,3.0,5h ago,2019-08-18T05:14:42.958Z,None,Maggie L,Fandango,8B9ED608-C9C6-4EAF-82A3-1DF9FFFD0EA4
8,2019-08-18T04:58:59.214Z,None,Bryan A,False,False,False,True,STAR_5,"Great, absolutely great! The animation was gr...",5.0,5h ago,2019-08-18T04:58:59.214Z,None,Bryan A,Fandango,C792B3A4-EC33-48C2-A911-A2AAB946B94F
9,2019-08-18T04:51:44.361Z,None,Bethany,False,False,False,True,STAR_5,It was just like the first one. Very good anim...,5.0,5h ago,2019-08-18T04:51:44.361Z,None,Bethany,Fandango,45c8dfaa-e20f-44df-8f01-445c0f055e42


In [7]:
movie_df.isnull().sum()

createDate             0
displayImageUrl     2778
displayName          138
hasProfanity           0
hasSpoilers            0
isSuperReviewer        0
isVerified             0
rating                 0
review                 0
score                  0
timeFromCreation       0
updateDate             0
user.accountLink    2390
user.displayName     138
user.realm             0
user.userId            0
dtype: int64

In [8]:
message="Review column and score column is not having null values. This is the column which we can use as per conditions. Hence, no need of impution."
print_line="----------------------------------------------------------------------------------------------------------------------------"
print(print_line)
print(message)
print(print_line)
message=""

----------------------------------------------------------------------------------------------------------------------------
Review column and score column is not having null values. This is the column which we can use as per conditions. Hence, no need of impution.
----------------------------------------------------------------------------------------------------------------------------


### Reading Ultimate Test data on which final prediction is to be done

In [9]:
Ultimate_TestData = pd.read_csv("../test-1566619745327.csv")

In [10]:
Ultimate_TestData.head()

,ReviewID,review
0,92876,Was good. Nothing like the original but I beli...
1,92877,I absolutely loved it! A wonderful rendition o...
2,92878,I love the movie! Good job director! \nI appre...
3,92879,"GREAT MOVIE!! Same as the original, but the c..."
4,92880,Realistic. Fantastic special effects


In [11]:
Ultimate_TestData.drop('ReviewID',inplace=True,axis=1)

In [12]:
Ultimate_TestData.head()

,review
0,Was good. Nothing like the original but I beli...
1,I absolutely loved it! A wonderful rendition o...
2,I love the movie! Good job director! \nI appre...
3,"GREAT MOVIE!! Same as the original, but the c..."
4,Realistic. Fantastic special effects


In [13]:
Ultimate_TestData.isnull().sum()

review    0
dtype: int64

In [14]:
message="We don't see any Review as null. Hence, we are all good."
print(print_line)
print(message)
print(print_line)
message=""

----------------------------------------------------------------------------------------------------------------------------
We don't see any Review as null. Hence, we are all good.
----------------------------------------------------------------------------------------------------------------------------


### Add the target variable for classification to input Data

In [15]:
movie_df['targetSentiment']=[0 if x>3 else 1 for x in movie_df['score']]

### Dropping the columns which deemed to be ignored as per instructions

In [16]:
movie_df.columns

Index(['createDate', 'displayImageUrl', 'displayName', 'hasProfanity',
       'hasSpoilers', 'isSuperReviewer', 'isVerified', 'rating', 'review',
       'score', 'timeFromCreation', 'updateDate', 'user.accountLink',
       'user.displayName', 'user.realm', 'user.userId', 'targetSentiment'],
      dtype='object')

In [17]:
columns_except_review_target = ['createDate', 'displayImageUrl', 'displayName', 'hasProfanity',
       'hasSpoilers', 'isSuperReviewer', 'isVerified', 'rating','timeFromCreation', 'updateDate', 'user.accountLink',
       'user.displayName', 'user.realm', 'user.userId','score']

In [18]:
movie_df.drop(columns_except_review_target,inplace=True,axis=1)

In [19]:
movie_df.head()

,review,targetSentiment
0,I liked most that the animation made the anima...,0
1,Amazing! So realistic and incredible music,0
2,Classic. Good remake. Loved it. Glover was out...,0
3,Nice animation/CGI but completely lacking the ...,1
4,Good but go mainly for the sentimental value,0


In [20]:
movie_df.tail()

,review,targetSentiment
2995,I took my 8 year old grandson. We both enjoyed...,0
2996,The animated version is still number one. All...,1
2997,Wonderful movie. I love this movie with the re...,0
2998,My 3 stars is for the special effects alone. I...,1
2999,By trying to be realistic the movie loses its ...,1


## Text processing

### Functions for Text cleaning

In [21]:
# Function to do the contraction operation
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)

    return expanded_text


# Function to do normalize operation
def normalize_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #https://docs.python.org/2/library/unicodedata.html
    return text


# Function to add tag in sentences having did not, could not etc till punctuation
def add_splTAG_after_not(sentence=None):
    transformed = re.sub(r'\b(?:not|never)\b[\w\s]+[^\w\s]',
       lambda match: re.sub(r'(\s+)(\w+)', r'\1negtagneg\2', match.group(0)), 
       sentence,
       flags=re.IGNORECASE)
    return transformed


# Function to transform the word to its lemma form
def tokenize_lemma_clean(dataframe,column_name,dest_column_name,StopWord_Removal=False,Only_2Len_word_Removal=False):
    # text cleaning and pre-processing   
    for index,row in dataframe.iterrows():  
        row[column_name]=row[column_name].lower()

        doc=nlp(row[column_name])

        ## 1st pass of removing only 2 letter words or stop words
        if(Only_2Len_word_Removal):
            #may use STOP words for conjuction detection
            #remove words with <=2 chars
            clean_tokens1 = [token for token in doc if len(token.text)>2]
            
        if(StopWord_Removal):
        # remove stop words
            if(Only_2Len_word_Removal):
                clean_tokens1 = [token for token in clean_tokens1 if not token.is_stop]
            else:
                clean_tokens1 = [token for token in doc if not token.is_stop]
       
        ## 2nd pass of removing non-alpha words 
        if ((StopWord_Removal==True) or ( Only_2Len_word_Removal==True)):
            clean_tokens2_bool = [token.is_alpha for token in clean_tokens1]
            #clean_tokens2_bool = [token.is_alpha for token in doc]
            clean_tokens2=list(compress(clean_tokens1,clean_tokens2_bool))
        else:
            ## if only Lemma operation is needed
            clean_tokens2 = [token for token in doc]
        
        ## 3rd pass of detecting special NOT tag added earlier for did not type of sentences
        ## Where ever encounter the spl tag addtition, keep it in the review comment for later processing
        ## Hence ignore it from Lemma operation
        ## Also for the added word in spltag, keep the lemma word, so that its easier to create a dictionary 
        ## of antonymns rather than having issues with past or future tense of same words/verbs
        not_string = re.compile('negtagneg.*')
        
        clean_tokens3=[]
        for token in clean_tokens2:
            if(not_string.match(token.text)):
                neg_whole_word=""
                neg_whole_word=token.text
                word_for_lemma = neg_whole_word.split('negtagneg')
                neg_doc = nlp(word_for_lemma[1])
                neg_lemma_word=""
                for tkn in neg_doc:
                    if (len(tkn)>2): #remove 2 letter words, which will be stopwords added with negation
                        neg_lemma_word = tkn.lemma_
                
                clean_tokens3.append("negtagneg"+neg_lemma_word)
            else:
                clean_tokens3.append(token.lemma_)

        clean_text=' '.join(clean_tokens3)

        ## Add the cleaned text to the dataframe column desired
        #dataframe.at[index,column_name]=clean_text
        #print("clean text:", clean_text)
        dataframe.at[index,dest_column_name]=clean_text

        
# Function to keep specific POS tag words 
# whichever POS tag words are needed can be passed in X1 ...X4
def pos_to_keep_and_count(dataframe,column_name,dest_column_name,dest_count_coulmn_name,Take_Count=False, x1=None,x2=None,x3=None,x4=None):

    for index,row in dataframe.iterrows():
        doc=nlp(row[column_name])
        
        ## words which have different POS tags based on context / placement
        ambiguous_pos_words = ['like','love']

        # POS specific words
        pos_tokens = [token.text for token in doc if ((token.pos_ in [x1,x2,x3,x4])or(token.text in ambiguous_pos_words ))]
        

        pos_text=' '.join(pos_tokens)
        
        if(Take_Count):
            # add # of POS tags in desired column nameS
            dataframe.at[index,dest_count_coulmn_name]=len(pos_tokens)

        dataframe.at[index,dest_column_name]=pos_text

        
# Change to antonyms for words suceeding the NOT spl tags        
def antonyms_change(dataframe,column_name,dest_column_name,word=None,antonym=None):
    # text cleaning and pre-processing
    #print("coming inside antonymns")
    for index,row in dataframe.iterrows():

        doc=row[column_name].split()

        not_string = re.compile("^"+word+"$")

        reverse_tokens=[]

        for token in doc:
            if(not_string.match(token)):
                if(len(antonym)>1):
                    reverse_tokens.append(antonym)
            else:
                reverse_tokens.append(token)

        clean_text=' '.join(reverse_tokens)

        dataframe.at[index,dest_column_name]=clean_text


In [22]:
# Dictionary list of all the negations replacement
all_antonymns_dict = {
'negtagneglike':'dislike',
'negtagneggood':'bad',
'negtagnegoriginal':'fake',
'negtagnegremember':'forget',
'negtagnegnear':'far',
'negtagnegfeel':'apathy',
'negtagnegadd':'miss',
'negtagnegkeep':'remove',
'negtagnegthink':'forget',
'negtagnegjustice':'injustice',
'negtagnegadd':'diminish',
'negtagnegway':'away',
'negtagnegmake':'destroy',
'negtagnegunlike':'unlike',
'negtagnegunnecessary':'unnecessary',
'negtagnegamazing': 'boring',
'negtagneganimation': 'mockery',
'negtagnegcartoon': 'spoof',
'negtagnegbelieve': 'disbelieve',
'negtagnegbetter': 'worse',
'negtagnegbig':'small',
'negtagnegcare':'disregard',
'negtagnegbad': 'bad',
'negtagnegbring':'repulse',
'negtagnegbuy':'relinquish',
'negtagneglion':'',
'negtagnegking':'',
'negtagnegall':'',
'negtagnegas':'',
'negtagnegand':'',
'negtagnega':'',
'negtagnegbe':'',
'negtagnegbut':'',
'negtagnegfor':'',
'negtagneghave':'',
'negtagnegi':'',
'negtagnegis':'',
'negtagnegin':'',
'negtagnegabove':'',
'negtagnegit':'',
'negtagnegmovie':'',
'negtagnegmuch':'',
'negtagnegnot':'',
'negtagnegof':'',
'negtagnegon':'',
'negtagnegthe':'',
'negtagnegthat':'',
'negtagnegthis':'',
'negtagnegwas':'',
'negtagnegis':'',
'negtagnegare':'',
'negtagnegwith':'',
'negtagnegof':'',
'negtagnegif':'',
'negtagnegversion':'',
'negtagnegneed':'',
'negtagnegremake':'',
'negtagnegput':'',
'negtagnegnala':'',
'negtagnegme':'',
'negtagnegbeyonce':'',
'negtagneganything':'',
'negtagnegbeing':'',
'negtagneg-PRON-':'',
'negtagnegdo':'',
'negtagneg':'',
'negtagnegsome':'',
'negtagnegstuff':'',
'negtagnegthing':'',
'negtagnegout':'',
'negtagneganimal':'',
'negtagneganimate': '',
'negtagnegfilm': '',
'negtagnegany':'',
'negnegtagneganyone':'',
'negtagnegattention':'',
'negtagnegaudience':'',
'negtagnegbeauty':'',
'negtagnegbecause':'',
'negtagnegbefore':'',
'negtagnegbegin':'',
'negtagnegthan':'',
'negtagnegbother':'',
'negtagnegcan':'',


} ## TODO: will add more negative words for replacement


# substitute words 
# will be using for changing spltagged words for negatives to antonymns
def substitute_words(dataframe,source_column_name,dest_column_name,dict_rep_words=all_antonymns_dict):
    try:
        dataframe.insert(2,source_column_name,dataframe[clean_after_lemma_n_stop_2len_word])
        print ("Creating a column for the cleaned antonyms process")
    except:
        print ("Error: May be the column already exits")

    print("Negation replacement in progress ...")
        
    for key in dict_rep_words.keys():
        antonyms_change(dataframe,source_column_name,dest_column_name,key,dict_rep_words[key])
        
    print("Negation replacement all Done!")  

### Functions for Feature Extraction (non-lexical) from review comments

In [23]:
# Extract Features from the review content 
# #Words
# #Capital Words
# #Question Marks 
# #Exclaimation Marks
# #NOT words
# #Emojis

class Spl_Chars_Counts(BaseEstimator, TransformerMixin):
    
    def count_regex(self, pattern, tweet):
        return len(re.findall(pattern, tweet,flags=re.IGNORECASE))

    def count_CAPS_regex(self, pattern, tweet):
        return len(re.findall(pattern, tweet))
    
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        count_words = X.apply(lambda x: self.count_regex(r'\w+', x)) 
        count_love = X.apply(lambda x: self.count_regex(r'love|great|superb|amazing|awesome|beautiful|fantastic|marvellous|magnificent|stunning|spectacular|wonderful|remarkable|fabulous|incredible|astounding|astonishing|unbelievable', x))
        count_shit = X.apply(lambda x: self.count_regex(r'hate|dislik|ugly|rubbish|trash|garbage|awful|boring|slow|waste|weird|forgettable|distast|lack|shit|ass|dung|fuck|dick|cunt|bitch|bastard|ruin', x))
        count_capital_words = X.apply(lambda x: self.count_CAPS_regex(r'\b[A-Z]{2,}\b', x))
        count_quest_marks = X.apply(lambda x: self.count_regex(r'\?', x))
        count_excl_marks = X.apply(lambda x: self.count_regex(r'!', x))
        count_NOT_words = X.apply(lambda x: self.count_regex(r'\bN*oo*t*\b', x))
        count_emojis = X.apply(lambda x: emoji.demojize(x)).apply(lambda x: self.count_regex(r':[a-z_&]+:', x))
        
        df = pd.DataFrame({
                           'count_words': count_words
                           , 'count_capital_words': count_capital_words
                           , 'count_love' : count_love
                           , 'count_shit' : count_shit
                           , 'count_quest_marks': count_quest_marks
                           , 'count_excl_marks': count_excl_marks
                           , 'count_NOT_words': count_NOT_words
                           , 'count_emojis': count_emojis
                          })
        
        return df

### Function to check polarity of words/ phrases/ sentences

In [24]:
def review_polarity(dataframe,column_name,dest_column_name,MULT_Factor):
    for index,row in dataframe.iterrows():
        review=TextBlob(row[column_name])

        dataframe.at[index,dest_column_name]=MULT_Factor*review.polarity


### Function to calculate average of w2v for list of words

In [25]:
def compute_avg_w2v_vector(w2v_dict, review):
    list_of_word_vectors = [w2v_dict[w] for w in review if w in w2v_dict.vocab.keys()]
    
    if len(list_of_word_vectors) == 0:
        result = [0.0]*SIZE
    else:
        result = np.sum(list_of_word_vectors, axis=0) / len(list_of_word_vectors)
        
    return result


## Starting Processing of Text 

In [26]:
#### copying train data till this moment and safe-keeping; Using the copied variable for processing ahead 

In [27]:
total_data = movie_df.copy(deep=True)

In [28]:
#columns name for orginal review text
review = 'review'
review_polar = 'review_polar'

## for 1st stage of cleaning data
clean_spll_contr_accnt_NOT = 'spll_contr_accnt_NOT'

## for 2nd stage of cleaning data
clean_after_lemma_only = 'lemma_only'
clean_after_lemma_n_stop = 'lemma_stop'
clean_after_lemma_only_2len_word = 'lemma_2len_word'
clean_after_lemma_n_stop_2len_word = 'lemma_stop_2len_word'

## for 3rd stage of cleaning data
clean_after_antonymns = 'antonymns_upd'

## for POS tags
adj_count = 'adj_count'
adj_polar ='adj_polar'
adj_text='adj_text'


adv_verb_count = 'adv_verb_count'
adv_verb_polar = 'adv_verb_polar'
adv_verb_text = 'adv_verb_text'


## target column
targetSentiment = 'targetSentiment'

In [29]:
total_data.head()

,review,targetSentiment
0,I liked most that the animation made the anima...,0
1,Amazing! So realistic and incredible music,0
2,Classic. Good remake. Loved it. Glover was out...,0
3,Nice animation/CGI but completely lacking the ...,1
4,Good but go mainly for the sentimental value,0


### 1. Spell correction

In [30]:
#total_data[clean_spll_contr_accnt_NOT] = [TextBlob(text).correct() for text in total_data['review']]

## TextBlob Spell correction changing many words into different words, which is not desired.
## Hence not using it here

### 2. Expand contractions

In [31]:
total_data[clean_spll_contr_accnt_NOT] = [expand_contractions(re.sub('’', "'", text)) for text in total_data['review']]

## Repeat the same for the final test data
Ultimate_TestData[clean_spll_contr_accnt_NOT] = [expand_contractions(re.sub('’', "'", text)) for text in Ultimate_TestData['review']]

In [32]:
total_data.head()

,review,targetSentiment,spll_contr_accnt_NOT
0,I liked most that the animation made the anima...,0,I liked most that the animation made the anima...
1,Amazing! So realistic and incredible music,0,Amazing! So realistic and incredible music
2,Classic. Good remake. Loved it. Glover was out...,0,Classic. Good remake. Loved it. Glover was out...
3,Nice animation/CGI but completely lacking the ...,1,Nice animation/CGI but completely lacking the ...
4,Good but go mainly for the sentimental value,0,Good but go mainly for the sentimental value


In [33]:
Ultimate_TestData.head()

,review,spll_contr_accnt_NOT
0,Was good. Nothing like the original but I beli...,Was good. Nothing like the original but I beli...
1,I absolutely loved it! A wonderful rendition o...,I absolutely loved it! A wonderful rendition o...
2,I love the movie! Good job director! \nI appre...,I love the movie! Good job director! \nI appre...
3,"GREAT MOVIE!! Same as the original, but the c...","GREAT MOVIE!! Same as the original, but the c..."
4,Realistic. Fantastic special effects,Realistic. Fantastic special effects


### 3. Normalize accented characters

In [34]:
total_data[clean_spll_contr_accnt_NOT] = [normalize_accented_chars(text) for text in total_data[clean_spll_contr_accnt_NOT]]

## Repeat the same for the final test data
Ultimate_TestData[clean_spll_contr_accnt_NOT] = [normalize_accented_chars(text) for text in Ultimate_TestData[clean_spll_contr_accnt_NOT]]

In [35]:
total_data.head()

,review,targetSentiment,spll_contr_accnt_NOT
0,I liked most that the animation made the anima...,0,I liked most that the animation made the anima...
1,Amazing! So realistic and incredible music,0,Amazing! So realistic and incredible music
2,Classic. Good remake. Loved it. Glover was out...,0,Classic. Good remake. Loved it. Glover was out...
3,Nice animation/CGI but completely lacking the ...,1,Nice animation/CGI but completely lacking the ...
4,Good but go mainly for the sentimental value,0,Good but go mainly for the sentimental value


### 4. Add NOT_ after not is encountered till punctuation

In [36]:
total_data[clean_spll_contr_accnt_NOT] = [add_splTAG_after_not(text) for text in total_data[clean_spll_contr_accnt_NOT]]

## Repeat the same for the final test data
Ultimate_TestData[clean_spll_contr_accnt_NOT] = [add_splTAG_after_not(text) for text in Ultimate_TestData[clean_spll_contr_accnt_NOT]]


In [37]:
total_data.head()

,review,targetSentiment,spll_contr_accnt_NOT
0,I liked most that the animation made the anima...,0,I liked most that the animation made the anima...
1,Amazing! So realistic and incredible music,0,Amazing! So realistic and incredible music
2,Classic. Good remake. Loved it. Glover was out...,0,Classic. Good remake. Loved it. Glover was out...
3,Nice animation/CGI but completely lacking the ...,1,Nice animation/CGI but completely lacking the ...
4,Good but go mainly for the sentimental value,0,Good but go mainly for the sentimental value


### 5. Do lemma and cleaning of stop words etc

In [38]:
tokenize_lemma_clean(total_data,clean_spll_contr_accnt_NOT,clean_after_lemma_n_stop_2len_word,StopWord_Removal=True,Only_2Len_word_Removal=False)

## Repeat the same for the final test data
tokenize_lemma_clean(Ultimate_TestData,clean_spll_contr_accnt_NOT,clean_after_lemma_n_stop_2len_word,StopWord_Removal=True,Only_2Len_word_Removal=False)

In [39]:
total_data.head()

,review,targetSentiment,spll_contr_accnt_NOT,lemma_stop_2len_word
0,I liked most that the animation made the anima...,0,I liked most that the animation made the anima...,like animation animal look real
1,Amazing! So realistic and incredible music,0,Amazing! So realistic and incredible music,amazing realistic incredible music
2,Classic. Good remake. Loved it. Glover was out...,0,Classic. Good remake. Loved it. Glover was out...,classic good remake love glover outstanding
3,Nice animation/CGI but completely lacking the ...,1,Nice animation/CGI but completely lacking the ...,nice animation cgi completely lack disney styl...
4,Good but go mainly for the sentimental value,0,Good but go mainly for the sentimental value,good mainly sentimental value


### Replace words succeeding the NOT tags with Antonyms or remove if useless

In [40]:
## Before going ahead with identifying POS words, change the negation succeeding words to antonymns

In [41]:
substitute_words(total_data,source_column_name=clean_after_antonymns,dest_column_name=clean_after_antonymns,dict_rep_words=all_antonymns_dict)

## Repeat the same for the final test data
substitute_words(Ultimate_TestData,source_column_name=clean_after_antonymns,dest_column_name=clean_after_antonymns,dict_rep_words=all_antonymns_dict)


Creating a column for the cleaned antonyms process
Negation replacement in progress ...
Negation replacement all Done!
Creating a column for the cleaned antonyms process
Negation replacement in progress ...
Negation replacement all Done!


In [42]:
total_data.head()


,review,targetSentiment,antonymns_upd,spll_contr_accnt_NOT,lemma_stop_2len_word
0,I liked most that the animation made the anima...,0,like animation animal look real,I liked most that the animation made the anima...,like animation animal look real
1,Amazing! So realistic and incredible music,0,amazing realistic incredible music,Amazing! So realistic and incredible music,amazing realistic incredible music
2,Classic. Good remake. Loved it. Glover was out...,0,classic good remake love glover outstanding,Classic. Good remake. Loved it. Glover was out...,classic good remake love glover outstanding
3,Nice animation/CGI but completely lacking the ...,1,nice animation cgi completely lack disney styl...,Nice animation/CGI but completely lacking the ...,nice animation cgi completely lack disney styl...
4,Good but go mainly for the sentimental value,0,good mainly sentimental value,Good but go mainly for the sentimental value,good mainly sentimental value


In [43]:
Ultimate_TestData.head()

,review,spll_contr_accnt_NOT,antonymns_upd,lemma_stop_2len_word
0,Was good. Nothing like the original but I beli...,was good. nothing like the original but i beli...,good like original believe point,good like original believe point
1,I absolutely loved it! A wonderful rendition o...,i absolutely loved it! a wonderful rendition o...,absolutely love wonderful rendition original n...,absolutely love wonderful rendition original n...
2,I love the movie! Good job director! \nI appre...,i love the movie! good job director! \ni appre...,love movie good job director appreciate work g...,love movie good job director appreciate work g...
3,"GREAT MOVIE!! Same as the original, but the c...","great movie!! same as the original, but the c...",great movie original cinematography huge diffe...,great movie original cinematography huge diffe...
4,Realistic. Fantastic special effects,realistic. fantastic special effects,realistic fantastic special effect,realistic fantastic special effect


In [44]:
if 1==1:
    total_data.drop([clean_after_lemma_n_stop_2len_word,clean_spll_contr_accnt_NOT],axis=1,inplace=True)
    Ultimate_TestData.drop([clean_after_lemma_n_stop_2len_word,clean_spll_contr_accnt_NOT],axis=1,inplace=True)

In [45]:
total_data.head()

,review,targetSentiment,antonymns_upd
0,I liked most that the animation made the anima...,0,like animation animal look real
1,Amazing! So realistic and incredible music,0,amazing realistic incredible music
2,Classic. Good remake. Loved it. Glover was out...,0,classic good remake love glover outstanding
3,Nice animation/CGI but completely lacking the ...,1,nice animation cgi completely lack disney styl...
4,Good but go mainly for the sentimental value,0,good mainly sentimental value


In [46]:
Ultimate_TestData.head()

,review,antonymns_upd
0,Was good. Nothing like the original but I beli...,good like original believe point
1,I absolutely loved it! A wonderful rendition o...,absolutely love wonderful rendition original n...
2,I love the movie! Good job director! \nI appre...,love movie good job director appreciate work g...
3,"GREAT MOVIE!! Same as the original, but the c...",great movie original cinematography huge diffe...
4,Realistic. Fantastic special effects,realistic fantastic special effect


## Feature engineering

### 1. Get the POS tags and counts 

#### 1.1 Getting ADJ tags and counts

In [47]:
#pos_to_keep_and_count(dataframe,column_name,dest_column_name,dest_count_coulmn_name,Take_Count=False,
pos_to_keep_and_count(dataframe=total_data,column_name=clean_after_antonymns,dest_column_name=adj_text,dest_count_coulmn_name=adj_count,Take_Count=True,x1='ADJ')

## Repeat the same for the final test data
pos_to_keep_and_count(dataframe=Ultimate_TestData,column_name=clean_after_antonymns,dest_column_name=adj_text,dest_count_coulmn_name=adj_count,Take_Count=True,x1='ADJ')

In [48]:
total_data.head()

,review,targetSentiment,antonymns_upd,adj_count,adj_text
0,I liked most that the animation made the anima...,0,like animation animal look real,2.0,like real
1,Amazing! So realistic and incredible music,0,amazing realistic incredible music,3.0,amazing realistic incredible
2,Classic. Good remake. Loved it. Glover was out...,0,classic good remake love glover outstanding,4.0,classic good love outstanding
3,Nice animation/CGI but completely lacking the ...,1,nice animation cgi completely lack disney styl...,5.0,nice humour original undue real
4,Good but go mainly for the sentimental value,0,good mainly sentimental value,2.0,good sentimental


#### 1.2 Getting ADV,VERB tags and counts

In [49]:
pos_to_keep_and_count(dataframe=total_data,column_name=clean_after_antonymns,dest_column_name=adv_verb_text,dest_count_coulmn_name=adv_verb_count,Take_Count=True,x1='ADV',x2='VERB')

## Repeat the same for the final test data
pos_to_keep_and_count(dataframe=Ultimate_TestData,column_name=clean_after_antonymns,dest_column_name=adv_verb_text,dest_count_coulmn_name=adv_verb_count,Take_Count=True,x1='ADV',x2='VERB')

In [50]:
total_data.head()

,review,targetSentiment,antonymns_upd,adj_count,adj_text,adv_verb_count,adv_verb_text
0,I liked most that the animation made the anima...,0,like animation animal look real,2.0,like real,2.0,like look
1,Amazing! So realistic and incredible music,0,amazing realistic incredible music,3.0,amazing realistic incredible,0.0,
2,Classic. Good remake. Loved it. Glover was out...,0,classic good remake love glover outstanding,4.0,classic good love outstanding,1.0,love
3,Nice animation/CGI but completely lacking the ...,1,nice animation cgi completely lack disney styl...,5.0,nice humour original undue real,2.0,completely lack
4,Good but go mainly for the sentimental value,0,good mainly sentimental value,2.0,good sentimental,1.0,mainly


## 2. Getting polarity of review comments, adj text, adv-verb text

In [51]:
#review_polarity(dataframe,column_name,dest_column_name,MULT_Factor)


In [52]:
review_polarity(total_data,review,review_polar,1)

## Repeat the same for the final test data
review_polarity(Ultimate_TestData,review,review_polar,1)

In [53]:
review_polarity(total_data,adj_text,adj_polar,1)

## Repeat the same for the final test data
review_polarity(Ultimate_TestData,adj_text,adj_polar,1)

In [54]:
review_polarity(total_data,adv_verb_text,adv_verb_polar,1)

## Repeat the same for the final test data
review_polarity(Ultimate_TestData,adv_verb_text,adv_verb_polar,1)

In [55]:
total_data.head()

,review,targetSentiment,antonymns_upd,adj_count,adj_text,adv_verb_count,adv_verb_text,review_polar,adj_polar,adv_verb_polar
0,I liked most that the animation made the anima...,0,like animation animal look real,2.0,like real,2.0,like look,0.433333,0.200000,0.000000
1,Amazing! So realistic and incredible music,0,amazing realistic incredible music,3.0,amazing realistic incredible,0.0,,0.605556,0.555556,0.000000
2,Classic. Good remake. Loved it. Glover was out...,0,classic good remake love glover outstanding,4.0,classic good love outstanding,1.0,love,0.516667,0.466667,0.500000
3,Nice animation/CGI but completely lacking the ...,1,nice animation cgi completely lack disney styl...,5.0,nice humour original undue real,2.0,completely lack,0.318750,0.391667,0.100000
4,Good but go mainly for the sentimental value,0,good mainly sentimental value,2.0,good sentimental,1.0,mainly,0.205556,0.225000,0.166667


In [56]:
Ultimate_TestData.head()

,review,antonymns_upd,adj_count,adj_text,adv_verb_count,adv_verb_text,review_polar,adj_polar,adv_verb_polar
0,Was good. Nothing like the original but I beli...,good like original believe point,3.0,good like original,1.0,like,0.537500,0.537500,0.0
1,I absolutely loved it! A wonderful rendition o...,absolutely love wonderful rendition original n...,4.0,love wonderful original like,6.0,absolutely love not compare let like,0.712500,0.625000,0.5
2,I love the movie! Good job director! \nI appre...,love movie good job director appreciate work g...,3.0,love good great,2.0,love work,0.766667,0.666667,0.5
3,"GREAT MOVIE!! Same as the original, but the c...",great movie original cinematography huge diffe...,3.0,great original huge,0.0,,0.500000,0.525000,0.0
4,Realistic. Fantastic special effects,realistic fantastic special effect,3.0,realistic fantastic special,0.0,,0.307937,0.307937,0.0


## 3. Get the length, words and special chars presence from review text

In [57]:
tc = Spl_Chars_Counts()
df_extra_feat = tc.fit_transform(total_data[review])

## Repeat the same for the final test data
tc_FinalOut = Spl_Chars_Counts()
df_extra_feat_FinalOut = tc_FinalOut.fit_transform(Ultimate_TestData[review])

In [58]:
df_extra_feat.head()

,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,12,0,0,0,0,0,0,0
1,6,0,2,0,0,1,0,0
2,8,0,2,1,0,0,0,0
3,27,1,0,1,1,0,0,0
4,8,0,0,0,0,0,0,0


In [59]:
df_extra_feat_FinalOut.head()

,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,13,0,0,0,0,0,0,0
1,28,0,2,0,0,1,0,0
2,20,0,2,0,0,2,0,0
3,13,2,1,0,0,4,0,0
4,4,0,1,0,0,0,0,0


## 4. Make the final data frame after feature engineering

In [60]:
final_data = pd.concat([total_data,df_extra_feat],axis=1)

FinalOut_final_data = pd.concat([Ultimate_TestData,df_extra_feat_FinalOut],axis=1)

In [61]:
final_data.head()

,review,targetSentiment,antonymns_upd,adj_count,adj_text,adv_verb_count,adv_verb_text,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,I liked most that the animation made the anima...,0,like animation animal look real,2.0,like real,2.0,like look,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,Amazing! So realistic and incredible music,0,amazing realistic incredible music,3.0,amazing realistic incredible,0.0,,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,Classic. Good remake. Loved it. Glover was out...,0,classic good remake love glover outstanding,4.0,classic good love outstanding,1.0,love,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,Nice animation/CGI but completely lacking the ...,1,nice animation cgi completely lack disney styl...,5.0,nice humour original undue real,2.0,completely lack,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,Good but go mainly for the sentimental value,0,good mainly sentimental value,2.0,good sentimental,1.0,mainly,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0


In [62]:
FinalOut_final_data.head()

,review,antonymns_upd,adj_count,adj_text,adv_verb_count,adv_verb_text,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,Was good. Nothing like the original but I beli...,good like original believe point,3.0,good like original,1.0,like,0.537500,0.537500,0.0,13,0,0,0,0,0,0,0
1,I absolutely loved it! A wonderful rendition o...,absolutely love wonderful rendition original n...,4.0,love wonderful original like,6.0,absolutely love not compare let like,0.712500,0.625000,0.5,28,0,2,0,0,1,0,0
2,I love the movie! Good job director! \nI appre...,love movie good job director appreciate work g...,3.0,love good great,2.0,love work,0.766667,0.666667,0.5,20,0,2,0,0,2,0,0
3,"GREAT MOVIE!! Same as the original, but the c...",great movie original cinematography huge diffe...,3.0,great original huge,0.0,,0.500000,0.525000,0.0,13,2,1,0,0,4,0,0
4,Realistic. Fantastic special effects,realistic fantastic special effect,3.0,realistic fantastic special,0.0,,0.307937,0.307937,0.0,4,0,1,0,0,0,0,0


In [63]:
### removing unecessary text data columns : adj_text,  adv_verb_text

In [64]:
if 1==1:
    final_data.drop([adj_text,adv_verb_text,review],axis=1,inplace=True)
    FinalOut_final_data.drop([adj_text,adv_verb_text,review],axis=1,inplace=True)

In [65]:
final_data.head()

,targetSentiment,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,0,like animation animal look real,2.0,2.0,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,0,amazing realistic incredible music,3.0,0.0,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,0,classic good remake love glover outstanding,4.0,1.0,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,1,nice animation cgi completely lack disney styl...,5.0,2.0,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,0,good mainly sentimental value,2.0,1.0,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0


In [66]:
### 2 copies of data
### 1. for having non-lexical features
### 2. for having text + non-lexical features

In [67]:
final_data_nontext = final_data.drop([clean_after_antonymns],axis=1)
FinalOut_final_data_nontext = FinalOut_final_data.drop([clean_after_antonymns],axis=1)

In [68]:
final_data_text = final_data.copy(deep=True)
FinalOut_final_data_text = FinalOut_final_data.copy(deep=True)

In [69]:
FinalOut_final_data_text.head()

,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,good like original believe point,3.0,1.0,0.537500,0.537500,0.0,13,0,0,0,0,0,0,0
1,absolutely love wonderful rendition original n...,4.0,6.0,0.712500,0.625000,0.5,28,0,2,0,0,1,0,0
2,love movie good job director appreciate work g...,3.0,2.0,0.766667,0.666667,0.5,20,0,2,0,0,2,0,0
3,great movie original cinematography huge diffe...,3.0,0.0,0.500000,0.525000,0.0,13,2,1,0,0,4,0,0
4,realistic fantastic special effect,3.0,0.0,0.307937,0.307937,0.0,4,0,1,0,0,0,0,0


In [70]:
FinalOut_final_data_text.head()

,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,good like original believe point,3.0,1.0,0.537500,0.537500,0.0,13,0,0,0,0,0,0,0
1,absolutely love wonderful rendition original n...,4.0,6.0,0.712500,0.625000,0.5,28,0,2,0,0,1,0,0
2,love movie good job director appreciate work g...,3.0,2.0,0.766667,0.666667,0.5,20,0,2,0,0,2,0,0
3,great movie original cinematography huge diffe...,3.0,0.0,0.500000,0.525000,0.0,13,2,1,0,0,4,0,0
4,realistic fantastic special effect,3.0,0.0,0.307937,0.307937,0.0,4,0,1,0,0,0,0,0


## Data Frame to store all the results

In [71]:
model_summary = pd.DataFrame(columns=['Model_Name','Train_Accuracy','F1_0Class','F1_1Class','Test_Accuracy','F1_0Class_test','F1_1Class_test','Features','Comments'])
model_summary_index=0

In [72]:
model_summary

,Model_Name,Train_Accuracy,F1_0Class,F1_1Class,Test_Accuracy,F1_0Class_test,F1_1Class_test,Features,Comments


## ML models to be used

In [73]:
## Function for Logistic Regression

def logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name,TESTDATA=True):
    global model_summary_index
    for C in C_list:
        for penalty in Penalty:
            logisticRegr = LogisticRegression(penalty = penalty, C = C,random_state = 0)
            logisticRegr.fit(X_train_sm, y_train_sm)

            train_predicted_classes = logisticRegr.predict(X_train_sm)
            train_accuracy = accuracy_score(y_train_sm,train_predicted_classes)


            test_predicted_classes = logisticRegr.predict(X_test)

            train_accuracy = accuracy_score(y_train_sm,train_predicted_classes)
            train_f1_0 = f1_score(y_train_sm,train_predicted_classes,pos_label=0)
            train_f1_1 = f1_score(y_train_sm,train_predicted_classes,pos_label=1)
            

            
            
            print("-------------------------------------------------------------------------------------")
            print("C : ",C, "Penalty : ",penalty)
            print("TRAIN DATA ACCURACY",train_accuracy)
            print("\nTrain data f1-score for class '0'",train_f1_0)
            print("\nTrain data f1-score for class '1'",train_f1_1)
            
            model_summary.loc[model_summary_index,'Model_Name']='Logistic'
            model_summary.loc[model_summary_index,'Train_Accuracy']=train_accuracy

            model_summary.loc[model_summary_index,'F1_0Class']=train_f1_0
            model_summary.loc[model_summary_index,'F1_1Class']=train_f1_1

            if(TESTDATA==True):
                test_accuracy = accuracy_score(y_test,test_predicted_classes)
                test_f1_0 = f1_score(y_test,test_predicted_classes,pos_label=0)
                test_f1_1 = f1_score(y_test,test_predicted_classes,pos_label=1)
            
                print("TEST DATA ACCURACY",test_accuracy)
                print("\nTest data f1-score for class '0'",test_f1_0)
                print("\nTest data f1-score for class '1'",test_f1_1)

                model_summary.loc[model_summary_index,'Test_Accuracy'] = test_accuracy
                model_summary.loc[model_summary_index,'F1_0Class_test']=test_f1_0
                model_summary.loc[model_summary_index,'F1_1Class_test']=test_f1_1
                                     

            model_summary.loc[model_summary_index,'Features'] = Features_name
            model_summary.loc[model_summary_index,'Comments'] = " C : " + str(C) + "| Penalty : " + str(penalty)

            model_summary_index=model_summary_index+1
    
    #return the last one as default
    return logisticRegr

            
            

## Function for XGB model

def xgb_model(X_train_sm,y_train_sm,X_test,param_grid,Features_name,TESTDATA=True):
    global model_summary_index
    
    XGB = XGBClassifier(n_jobs=-1)
    CV_XGB = GridSearchCV(estimator=XGB, param_grid=param_grid, cv= 10)


    CV_XGB.fit(X_train_sm, y_train_sm)

    train_predictions_xgb =  CV_XGB.predict(X_train_sm)
    test_predictions_xgb =  CV_XGB.predict(X_test.values)
                                     
    train_accuracy = accuracy_score(y_train_sm,train_predictions_xgb)
    train_f1_0 = f1_score(y_train_sm,train_predictions_xgb,pos_label=0)
    train_f1_1 = f1_score(y_train_sm,train_predictions_xgb,pos_label=1)
 


    print("-------------------------------------------------------------------------------------")
    print("CV_XGB.best_score_ ",CV_XGB.best_score_)
    print("CV_XGB.best_params_ ",CV_XGB.best_params_)

    print("TRAIN DATA ACCURACY",train_accuracy)
    print("\nTrain data f1-score for class '0'",train_f1_0)
    print("\nTrain data f1-score for class '1'",train_f1_1)

                                     
    model_summary.loc[model_summary_index,'Model_Name']='XGBoost'
    model_summary.loc[model_summary_index,'Train_Accuracy']=train_accuracy

    model_summary.loc[model_summary_index,'F1_0Class']=train_f1_0
    model_summary.loc[model_summary_index,'F1_1Class']=train_f1_1
                                     
    if(TESTDATA==True):
                                     
        test_accuracy = accuracy_score(y_test,test_predictions_xgb)
        test_f1_0 = f1_score(y_test,test_predictions_xgb,pos_label=0)
        test_f1_1 = f1_score(y_test,test_predictions_xgb,pos_label=1)
        print("TEST DATA ACCURACY",test_accuracy)
        print("\nTest data f1-score for class '0'",test_f1_0)
        print("\nTest data f1-score for class '1'",test_f1_1)
                                     
        model_summary.loc[model_summary_index,'Test_Accuracy'] = test_accuracy
        model_summary.loc[model_summary_index,'F1_0Class_test']= test_f1_0
        model_summary.loc[model_summary_index,'F1_1Class_test']= test_f1_1

    model_summary.loc[model_summary_index,'Features'] = Features_name
    model_summary.loc[model_summary_index,'Comments'] = " Best_Score_ : " + str(CV_XGB.best_score_) + "| Best_Param_ : " + str(CV_XGB.best_params_)

    model_summary_index=model_summary_index+1
    
    return CV_XGB
    

## Function for RFC model
def rfc_model(X_train_sm,y_train_sm,X_test,param_grid,Features_name,TESTDATA=True):
    global model_summary_index
    
    rfc_grid = RandomForestClassifier(n_jobs=-1, max_features='sqrt')
    rfc_cv_grid = RandomizedSearchCV(estimator = rfc_grid, param_distributions = param_grid, cv = 3, n_iter=10)

    rfc_cv_grid.fit(X_train_sm, y_train_sm)

    train_predictions_rf = rfc_cv_grid.predict(X_train_sm)
    test_predictions_rf = rfc_cv_grid.predict(X_test)
                             
    train_accuracy = accuracy_score(y_train_sm,train_predictions_rf)
    train_f1_0 = f1_score(y_train_sm,train_predictions_rf,pos_label=0)
    train_f1_1 = f1_score(y_train_sm,train_predictions_rf,pos_label=1)


    print("-------------------------------------------------------------------------------------")
    print("rfc_cv_grid.best_score_ ",rfc_cv_grid.best_score_)
    print("rfc_cv_grid.best_params_ ",rfc_cv_grid.best_params_)

    print("TRAIN DATA ACCURACY",train_accuracy)
    print("\nTrain data f1-score for class '0'",train_f1_0)
    print("\nTrain data f1-score for class '1'",train_f1_1)
                             
    model_summary.loc[model_summary_index,'Model_Name']='RandomForest'
    model_summary.loc[model_summary_index,'Train_Accuracy']=train_accuracy

    model_summary.loc[model_summary_index,'F1_0Class']=train_f1_0
    model_summary.loc[model_summary_index,'F1_1Class']=train_f1_1

    if(TESTDATA==True):
        test_accuracy = accuracy_score(y_test,test_predictions_rf)
        test_f1_0 =f1_score(y_test,test_predictions_rf,pos_label=0)
        test_f1_1 =f1_score(y_test,test_predictions_rf,pos_label=1)
                             
        print("TEST DATA ACCURACY",test_accuracy)
        print("\nTest data f1-score for class '0'",test_f1_0)
        print("\nTest data f1-score for class '1'",test_f1_1)

        model_summary.loc[model_summary_index,'Test_Accuracy'] = test_accuracy
        model_summary.loc[model_summary_index,'F1_0Class_test']=test_f1_0
        model_summary.loc[model_summary_index,'F1_1Class_test']=test_f1_1

    model_summary.loc[model_summary_index,'Features'] = Features_name
    model_summary.loc[model_summary_index,'Comments'] = " Best_Score_ : " + str(rfc_cv_grid.best_score_) + "| Best_Param_ : " + str(rfc_cv_grid.best_params_)

    model_summary_index=model_summary_index+1  

    return rfc_cv_grid

## Model Building

## 1. Model building with non-text data

In [74]:
final_data_nontext.head()

,targetSentiment,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,0,2.0,2.0,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,0,3.0,0.0,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,0,4.0,1.0,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,1,5.0,2.0,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,0,2.0,1.0,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0


In [75]:
X = final_data_nontext.drop([targetSentiment], axis = 1)
y = y = final_data_nontext[targetSentiment]

#### train-test split

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

## For future use
X_train_ntext_fut = X_train.copy(deep=True)
X_test_ntext_fut = X_test.copy(deep=True)

y_train_ntext_fut = y_train.copy(deep=True)
y_test_ntext_fut = y_test.copy(deep=True)

#### SMOTE the train data as there is data imbalance

In [77]:
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)

In [78]:
print(f"X_train.shape = {X_train.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_train_sm.shape = {X_train_sm.shape}")
print(f"y_train_sm.shape = {y_train_sm.shape}")
print(f"sum(y_train_sm) = {sum(y_train_sm)}")

X_train.shape = (2400, 13)
y_train.shape = (2400,)
X_train_sm.shape = (3468, 13)
y_train_sm.shape = (3468,)
sum(y_train_sm) = 1734


### 1.1.1 Logistic Regression

In [79]:
C_list = [0.001,0.01,0.1,1,10,100]
Penalty = ['l1','l2']

logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name="Non-Text")



-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.0

Train data f1-score for class '1' 0.6666666666666666
TEST DATA ACCURACY 0.27

Test data f1-score for class '0' 0.0

Test data f1-score for class '1' 0.4251968503937008
-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l2
TRAIN DATA ACCURACY 0.7670126874279123

Train data f1-score for class '0' 0.7580838323353294

Train data f1-score for class '1' 0.7753058954393771
TEST DATA ACCURACY 0.785

Test data f1-score for class '0' 0.8401486988847583

Test data f1-score for class '1' 0.6717557251908397
-------------------------------------------------------------------------------------
C :  0.01 Penalty :  l1
TRAIN DATA ACCURACY 0.7615340253748558

Train data f1-score for class '0' 0.7562628941939286

Train data f1-score for class '1' 0.7665819926615862
TEST DATA A

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 1.1.2 XG Boost

In [80]:
# Use a grid over parameters of interest
param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),
    'n_estimators':[100],
    'max_depth': [10]
}

xgb_model(X_train_sm,y_train_sm,X_test,param_grid,"Non-Text")

C:\Users\trex\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-------------------------------------------------------------------------------------
CV_XGB.best_score_  0.8532295271049596
CV_XGB.best_params_  {'colsample_bytree': 0.7, 'max_depth': 10, 'n_estimators': 100}
TRAIN DATA ACCURACY 0.9824106113033448

Train data f1-score for class '0' 0.9824863623313235

Train data f1-score for class '1' 0.9823342021430639
TEST DATA ACCURACY 0.7916666666666666

Test data f1-score for class '0' 0.8564867967853041

Test data f1-score for class '1' 0.6200607902735563


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 1.1.3 RandomForest 

In [81]:
## Use a grid over parameters of interest
## n_estimators is the number of trees in the forest
## max_depth is how deep each tree can be
## min_sample_leaf is the minimum samples required in each leaf node for the root node to split
## "A node will only be split if in each of it's leaf nodes there should be min_sample_leaf"

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20],
           "class_weight" : ['balanced','balanced_subsample']}


rfc_model(X_train_sm,y_train_sm,X_test,param_grid,"Non-Text")

-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.8194925028835064
rfc_cv_grid.best_params_  {'n_estimators': 100, 'min_samples_leaf': 10, 'max_depth': 14, 'class_weight': 'balanced'}
TRAIN DATA ACCURACY 0.8664936562860438

Train data f1-score for class '0' 0.863139225539462

Train data f1-score for class '1' 0.8696875879538419
TEST DATA ACCURACY 0.795

Test data f1-score for class '0' 0.851985559566787

Test data f1-score for class '1' 0.6666666666666666


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [10, 25, 50, 75, 100], 'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced', 'balanced_subsample']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

## 2. Model building with Text data

## 2.1 Using TF-IDF only

In [82]:
final_data_text

,targetSentiment,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,0,like animation animal look real,2.0,2.0,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,0,amazing realistic incredible music,3.0,0.0,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,0,classic good remake love glover outstanding,4.0,1.0,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,1,nice animation cgi completely lack disney styl...,5.0,2.0,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,0,good mainly sentimental value,2.0,1.0,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0
5,0,animation good,1.0,0.0,0.700000,0.700000,0.000000,5,0,0,0,0,0,0,0
6,0,think awesone like originalaty enjoy nail wait...,2.0,5.0,0.333333,0.300000,0.400000,34,0,0,0,0,0,0,0
7,1,think bring original voice actor movie iconic ...,5.0,9.0,0.328306,0.182273,0.366667,78,0,1,0,0,0,0,0
8,0,great absolutely great animation great story l...,6.0,4.0,0.439063,0.452083,0.350000,30,0,4,0,0,1,0,0
9,0,like good animation cute movie,3.0,1.0,0.603333,0.600000,0.000000,13,0,0,0,0,0,0,0


In [83]:
X = pd.DataFrame(final_data_text[clean_after_antonymns])
y = final_data_text[targetSentiment]

### train-test split

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

### TfIdf vectorizer

In [85]:
# Taking both unigram and bigram
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=10000000,
                                 min_df=0.001,
                                 use_idf=True, ngram_range=(1,2))


tfidf_vectorizer.fit(X_train[clean_after_antonymns])

X_train_tfidf_matrix= tfidf_vectorizer.transform(X_train[clean_after_antonymns])
X_test_tfidf_matrix = tfidf_vectorizer.transform(X_test[clean_after_antonymns])



In [86]:
X_train_tfidf_matrix

<2400x2291 sparse matrix of type '<class 'numpy.float64'>'
	with 29678 stored elements in Compressed Sparse Row format>

In [87]:
X_test_tfidf_matrix

<600x2291 sparse matrix of type '<class 'numpy.float64'>'
	with 7327 stored elements in Compressed Sparse Row format>

In [88]:
print(tfidf_vectorizer.get_feature_names())

['able', 'absolute', 'absolutely', 'absolutely amazing', 'absolutely beautiful', 'absolutely love', 'absolutely stunning', 'accurate', 'achieve', 'achievement', 'act', 'acting', 'acting bad', 'acting feel', 'action', 'action adaptation', 'action film', 'action movie', 'action remake', 'action version', 'actor', 'actor play', 'actor sound', 'actor voice', 'actual', 'actual animal', 'actually', 'adaptation', 'adaptation original', 'add', 'add little', 'add new', 'addition', 'additionally', 'admission', 'admit', 'adorable', 'adore', 'adult', 'adult child', 'affect', 'africa', 'african', 'age', 'ago', 'agree', 'agree critic', 'aladdin', 'alike', 'alive', 'allow', 'alright', 'alteration', 'amazing', 'amazing animal', 'amazing animation', 'amazing bring', 'amazing cgi', 'amazing feel', 'amazing good', 'amazing graphic', 'amazing great', 'amazing job', 'amazing look', 'amazing love', 'amazing movie', 'amazing real', 'amazing realistic', 'amazing story', 'amazing visual', 'amazing voice', 'ama

#### Making tfidf dataframe

In [89]:
X_train_tfidf_df = pd.DataFrame(X_train_tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names())
X_test_tfidf_df  = pd.DataFrame(X_test_tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names())

In [90]:
## For future use
X_train_tfidf_fut = X_train_tfidf_df.copy(deep=True)
X_test_tfidf_fut = X_test_tfidf_df.copy(deep=True)

y_train_tfidf_fut = y_train.copy(deep=True)
y_test_tfidf_fut = y_test.copy(deep=True)

#### SMOTE the train data as there is data imbalance

In [91]:
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_sample(X_train_tfidf_df, y_train)

In [92]:
print(f"X_train_tfidf_df.shape = {X_train_tfidf_df.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_train_sm.shape = {X_train_sm.shape}")
print(f"y_train_sm.shape = {y_train_sm.shape}")
print(f"sum(y_train_sm) = {sum(y_train_sm)}")

X_train_tfidf_df.shape = (2400, 2291)
y_train.shape = (2400,)
X_train_sm.shape = (3468, 2291)
y_train_sm.shape = (3468,)
sum(y_train_sm) = 1734


### 2.1.1 Logistic Regression

In [93]:
X_test=X_test_tfidf_df

C=[0.001,0.01,0.1,1,10,100]
Penalty=['l1','l2']
        
logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name="TFIDF")

-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.6666666666666666

Train data f1-score for class '1' 0.0
TEST DATA ACCURACY 0.73

Test data f1-score for class '0' 0.8439306358381503

Test data f1-score for class '1' 0.0
-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l2
TRAIN DATA ACCURACY 0.8457324106113033

Train data f1-score for class '0' 0.8387096774193549

Train data f1-score for class '1' 0.8521691074882564
TEST DATA ACCURACY 0.7916666666666666

Test data f1-score for class '0' 0.8481166464155528

Test data f1-score for class '1' 0.6684350132625995
-------------------------------------------------------------------------------------
C :  0.01 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.6666666666666666

Train data f1-score for class '1' 0.0
TEST DATA ACCURACY 0.73

Tes

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 2.1.2 XG Boost

In [94]:
X_test = X_test_tfidf_df

param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),
    'n_estimators':[100],
    'max_depth': [10]
}

xgb_model(X_train_sm,y_train_sm,X_test,param_grid,"TFIDF")

-------------------------------------------------------------------------------------
CV_XGB.best_score_  0.8722606689734718
CV_XGB.best_params_  {'colsample_bytree': 0.7, 'max_depth': 10, 'n_estimators': 100}
TRAIN DATA ACCURACY 0.9472318339100346

Train data f1-score for class '0' 0.9459698848538529

Train data f1-score for class '1' 0.94843617920541
TEST DATA ACCURACY 0.8033333333333333

Test data f1-score for class '0' 0.8637413394919169

Test data f1-score for class '1' 0.6467065868263472


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 2.1.3 RandomForest 

In [95]:
X_test = X_test_tfidf_df

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20],
           "class_weight" : ['balanced','balanced_subsample']}


rfc_model(X_train_sm,y_train_sm,X_test,param_grid,"TFIDF")

-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.8154555940023068
rfc_cv_grid.best_params_  {'n_estimators': 100, 'min_samples_leaf': 10, 'max_depth': 14, 'class_weight': 'balanced_subsample'}
TRAIN DATA ACCURACY 0.8379469434832757

Train data f1-score for class '0' 0.8438888888888889

Train data f1-score for class '1' 0.8315347721822542
TEST DATA ACCURACY 0.8116666666666666

Test data f1-score for class '0' 0.8714448236632537

Test data f1-score for class '1' 0.6479750778816198


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [10, 25, 50, 75, 100], 'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced', 'balanced_subsample']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [96]:
model_summary.tail()

,Model_Name,Train_Accuracy,F1_0Class,F1_1Class,Test_Accuracy,F1_0Class_test,F1_1Class_test,Features,Comments
23,Logistic,0.978662,0.97835,0.978965,0.82,0.87234,0.694915,TFIDF,C : 10| Penalty : l2
24,Logistic,0.997982,0.997977,0.997986,0.8,0.859155,0.655172,TFIDF,C : 100| Penalty : l1
25,Logistic,0.997693,0.997688,0.997699,0.788333,0.849704,0.642254,TFIDF,C : 100| Penalty : l2
26,XGBoost,0.947232,0.94597,0.948436,0.803333,0.863741,0.646707,TFIDF,Best_Score_ : 0.8722606689734718| Best_Param_...
27,RandomForest,0.837947,0.843889,0.831535,0.811667,0.871445,0.647975,TFIDF,Best_Score_ : 0.8154555940023068| Best_Param_...


## 2.2 Using WordVec model for average

In [97]:
X = pd.DataFrame(final_data_text[clean_after_antonymns])
y = final_data_text[targetSentiment]

### train-test split

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [99]:
w2v_column = 'clean_wordlist'

X_train[w2v_column] = X_train[clean_after_antonymns].apply(lambda x : word_tokenize(x))
X_test[w2v_column]  = X_test[clean_after_antonymns].apply(lambda x : word_tokenize(x))

In [100]:
SIZE = 50

model_w2v = gensim.models.Word2Vec(X_train[w2v_column], min_count=1, size=SIZE, window=5, workers=4)

In [101]:
X_train_w2v = X_train[w2v_column].apply(lambda x: compute_avg_w2v_vector(model_w2v.wv, x))
X_test_w2v = X_test[w2v_column].apply(lambda x: compute_avg_w2v_vector(model_w2v.wv, x))

In [102]:
X_train_w2v = pd.DataFrame(X_train_w2v.values.tolist(), index= X_train.index)
X_test_w2v = pd.DataFrame(X_test_w2v.values.tolist(), index= X_test.index)

In [103]:
X_train_w2v.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
642,-0.297958,0.367489,-0.246626,-0.261584,-0.116709,0.280616,0.137772,-0.479472,-0.132771,0.028295,...,-0.008211,-0.422410,0.090620,-0.482276,0.428462,0.711160,-0.122156,0.069951,0.207081,0.292646
700,-0.461686,0.567431,-0.380894,-0.404393,-0.180663,0.430318,0.207844,-0.738862,-0.202810,0.041715,...,-0.010185,-0.652578,0.141439,-0.744272,0.660093,1.101355,-0.192409,0.106926,0.322452,0.451868
226,-0.554168,0.684185,-0.458047,-0.488944,-0.224279,0.517357,0.260497,-0.889284,-0.246376,0.054824,...,-0.012917,-0.784576,0.172305,-0.893701,0.792233,1.321756,-0.220718,0.129371,0.388404,0.540933
1697,-0.399104,0.486107,-0.331721,-0.348955,-0.158279,0.375257,0.181171,-0.635940,-0.175247,0.034743,...,-0.013815,-0.567014,0.115369,-0.646949,0.572402,0.948451,-0.166771,0.090506,0.277374,0.387498
1010,-0.327948,0.403215,-0.272527,-0.286013,-0.129128,0.305954,0.149250,-0.522563,-0.142242,0.030306,...,-0.008318,-0.463409,0.101330,-0.530869,0.470282,0.782044,-0.137204,0.077862,0.229054,0.322284


In [104]:
X_test_w2v.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
1801,-0.267044,0.327484,-0.222221,-0.236154,-0.105266,0.252438,0.124450,-0.427222,-0.116464,0.025179,...,-0.005985,-0.375505,0.081262,-0.430189,0.384863,0.636892,-0.112133,0.061763,0.184580,0.263856
1190,-0.594491,0.737889,-0.503517,-0.525562,-0.233656,0.560344,0.269425,-0.966638,-0.263995,0.053962,...,-0.017467,-0.854705,0.187708,-0.975666,0.867683,1.443067,-0.247890,0.137173,0.426502,0.596233
1817,-0.473416,0.584392,-0.394117,-0.421644,-0.186030,0.444900,0.217332,-0.759370,-0.208797,0.044663,...,-0.010287,-0.674155,0.146224,-0.771807,0.679518,1.129637,-0.192128,0.112850,0.325904,0.465747
251,-0.282063,0.346562,-0.235256,-0.249489,-0.112440,0.263695,0.129067,-0.451760,-0.123960,0.025461,...,-0.006473,-0.401139,0.085563,-0.456080,0.404799,0.675786,-0.118021,0.067148,0.198143,0.277213
2505,-0.229298,0.281370,-0.189301,-0.202505,-0.090381,0.213806,0.102547,-0.365893,-0.101267,0.018305,...,-0.006769,-0.325721,0.070710,-0.371205,0.329167,0.548971,-0.097601,0.052902,0.160980,0.223560


In [105]:
## For future use
X_train_w2v_fut = X_train_w2v.copy(deep=True)
X_test_w2v_fut = X_test_w2v.copy(deep=True)

y_train_w2v_fut = y_train.copy(deep=True)
y_test_w2v_fut = y_test.copy(deep=True)

#### SMOTE the train data as there is data imbalance

In [106]:
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_sample(X_train_w2v, y_train)

In [107]:
print(f"X_train_w2v.shape = {X_train_w2v.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_train_sm.shape = {X_train_sm.shape}")
print(f"y_train_sm.shape = {y_train_sm.shape}")
print(f"sum(y_train_sm) = {sum(y_train_sm)}")

X_train_w2v.shape = (2400, 50)
y_train.shape = (2400,)
X_train_sm.shape = (3468, 50)
y_train_sm.shape = (3468,)
sum(y_train_sm) = 1734


### 2.2.1 Logistic Regression

In [108]:
X_test=X_test_w2v

C=[0.001,0.01,0.1,1,10,100]
Penalty=['l1','l2']
        
logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name="Word2Vec")


-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.6666666666666666

Train data f1-score for class '1' 0.0
TEST DATA ACCURACY 0.73

Test data f1-score for class '0' 0.8439306358381503

Test data f1-score for class '1' 0.0
-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l2
TRAIN DATA ACCURACY 0.5155709342560554

Train data f1-score for class '0' 0.6681153694192018

Train data f1-score for class '1' 0.10352187833511205
TEST DATA ACCURACY 0.725

Test data f1-score for class '0' 0.8387096774193548

Test data f1-score for class '1' 0.06779661016949153
-------------------------------------------------------------------------------------
C :  0.01 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.6666666666666666

Train data f1-score for class '1' 0.0
TEST DATA ACCURACY 0.73

Test data f1-s

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 2.2.1 XG Boost

In [109]:
X_test = X_test_w2v

param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),
    'n_estimators':[100],
    'max_depth': [10]
}

xgb_model(X_train_sm,y_train_sm,X_test,param_grid,"Word2Vec")


-------------------------------------------------------------------------------------
CV_XGB.best_score_  0.8189158016147635
CV_XGB.best_params_  {'colsample_bytree': 0.5, 'max_depth': 10, 'n_estimators': 100}
TRAIN DATA ACCURACY 0.9971164936562861

Train data f1-score for class '0' 0.9971131639722864

Train data f1-score for class '1' 0.9971198156682027
TEST DATA ACCURACY 0.6766666666666666

Test data f1-score for class '0' 0.7785388127853882

Test data f1-score for class '1' 0.4012345679012346


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 2.2.1 RandomForest

In [110]:
X_test = X_test_w2v

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20],
           "class_weight" : ['balanced','balanced_subsample']}


rfc_model(X_train_sm,y_train_sm,X_test,param_grid,"Word2Vec")


-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.7909457900807382
rfc_cv_grid.best_params_  {'n_estimators': 50, 'min_samples_leaf': 5, 'max_depth': 18, 'class_weight': 'balanced'}
TRAIN DATA ACCURACY 0.9685697808535179

Train data f1-score for class '0' 0.9682862961885365

Train data f1-score for class '1' 0.9688482423549586
TEST DATA ACCURACY 0.6933333333333334

Test data f1-score for class '0' 0.7875288683602771

Test data f1-score for class '1' 0.44910179640718567


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [10, 25, 50, 75, 100], 'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced', 'balanced_subsample']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

## 2.3 Using CountVectorizer model

In [111]:
X = pd.DataFrame(final_data_text[clean_after_antonymns])
y = final_data_text[targetSentiment]

### train-test split

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

### CountVectorizer

In [113]:
# Taking both unigram and bigram
cvect = CountVectorizer(max_df=0.9, max_features=10000000,
                                 min_df=0.001, ngram_range=(1,2))

#cvect = CountVectorizer()
cvect.fit(X_train[clean_after_antonymns])

X_train_cvect_matrix= cvect.transform(X_train[clean_after_antonymns])
X_test_cvect_matrix = cvect.transform(X_test[clean_after_antonymns])

In [114]:
X_train_cvect_matrix

<2400x2291 sparse matrix of type '<class 'numpy.int64'>'
	with 29678 stored elements in Compressed Sparse Row format>

In [115]:
print(cvect.get_feature_names())

['able', 'absolute', 'absolutely', 'absolutely amazing', 'absolutely beautiful', 'absolutely love', 'absolutely stunning', 'accurate', 'achieve', 'achievement', 'act', 'acting', 'acting bad', 'acting feel', 'action', 'action adaptation', 'action film', 'action movie', 'action remake', 'action version', 'actor', 'actor play', 'actor sound', 'actor voice', 'actual', 'actual animal', 'actually', 'adaptation', 'adaptation original', 'add', 'add little', 'add new', 'addition', 'additionally', 'admission', 'admit', 'adorable', 'adore', 'adult', 'adult child', 'affect', 'africa', 'african', 'age', 'ago', 'agree', 'agree critic', 'aladdin', 'alike', 'alive', 'allow', 'alright', 'alteration', 'amazing', 'amazing animal', 'amazing animation', 'amazing bring', 'amazing cgi', 'amazing feel', 'amazing good', 'amazing graphic', 'amazing great', 'amazing job', 'amazing look', 'amazing love', 'amazing movie', 'amazing real', 'amazing realistic', 'amazing story', 'amazing visual', 'amazing voice', 'ama

#### Making CountVectorize matrix as DF

In [116]:
X_train_cvect_df = pd.DataFrame(X_train_cvect_matrix.toarray(), columns=cvect.get_feature_names())
X_test_cvect_df  = pd.DataFrame(X_test_cvect_matrix.toarray(), columns=cvect.get_feature_names())

In [117]:
X_test_cvect_df.head()


,able,absolute,absolutely,absolutely amazing,absolutely beautiful,absolutely love,absolutely stunning,accurate,achieve,achievement,...,year old,yes,young,young child,young kid,youth,yr,yr old,zazu,zimmer
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
## For future use
X_train_cvect_fut = X_train_cvect_df.copy(deep=True)
X_test_cvect_fut = X_test_cvect_df.copy(deep=True)

y_train_cvect_fut = y_train.copy(deep=True)
y_test_cvect_fut = y_test.copy(deep=True)

#### SMOTE the train data as there is data imbalance

In [119]:
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_sample(X_train_cvect_df, y_train)

In [120]:
print(f"X_train_cvect_df.shape = {X_train_cvect_df.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_train_sm.shape = {X_train_sm.shape}")
print(f"y_train_sm.shape = {y_train_sm.shape}")
print(f"sum(y_train_sm) = {sum(y_train_sm)}")

X_train_cvect_df.shape = (2400, 2291)
y_train.shape = (2400,)
X_train_sm.shape = (3468, 2291)
y_train_sm.shape = (3468,)
sum(y_train_sm) = 1734


### 2.3.1 Logistic Regression

In [121]:
X_test=X_test_cvect_df

C=[0.001,0.01,0.1,1,10,100]
Penalty=['l1','l2']
        
logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name="CountVect")

-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.6666666666666666

Train data f1-score for class '1' 0.0
TEST DATA ACCURACY 0.73

Test data f1-score for class '0' 0.8439306358381503

Test data f1-score for class '1' 0.0
-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l2
TRAIN DATA ACCURACY 0.7753748558246828

Train data f1-score for class '0' 0.7740063823614738

Train data f1-score for class '1' 0.7767268558326168
TEST DATA ACCURACY 0.73

Test data f1-score for class '0' 0.8048192771084336

Test data f1-score for class '1' 0.5621621621621622
-------------------------------------------------------------------------------------
C :  0.01 Penalty :  l1
TRAIN DATA ACCURACY 0.6882929642445214

Train data f1-score for class '0' 0.5888170406998859

Train data f1-score for class '1' 0.7490132342697933
TEST DATA AC

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 2.3.1 XG Boost

In [122]:
X_test = X_test_cvect_df

param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),
    'n_estimators':[100],
    'max_depth': [10]
}

xgb_model(X_train_sm,y_train_sm,X_test,param_grid,"CountVect")



-------------------------------------------------------------------------------------
CV_XGB.best_score_  0.8281430219146482
CV_XGB.best_params_  {'colsample_bytree': 0.7, 'max_depth': 10, 'n_estimators': 100}
TRAIN DATA ACCURACY 0.9008073817762399

Train data f1-score for class '0' 0.894413750767342

Train data f1-score for class '1' 0.9064709081022294
TEST DATA ACCURACY 0.7516666666666667

Test data f1-score for class '0' 0.8219832735961768

Test data f1-score for class '1' 0.5895316804407715


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 2.3.1 RandomForest

In [123]:
X_test = X_test_cvect_df

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20],
           "class_weight" : ['balanced','balanced_subsample']}


rfc_model(X_train_sm,y_train_sm,X_test,param_grid,"CountVect") 


-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.7773933102652826
rfc_cv_grid.best_params_  {'n_estimators': 50, 'min_samples_leaf': 5, 'max_depth': 20, 'class_weight': 'balanced'}
TRAIN DATA ACCURACY 0.817762399077278

Train data f1-score for class '0' 0.8057775046097111

Train data f1-score for class '1' 0.828354155350353
TEST DATA ACCURACY 0.71

Test data f1-score for class '0' 0.7893462469733656

Test data f1-score for class '1' 0.53475935828877


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [10, 25, 50, 75, 100], 'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced', 'balanced_subsample']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

## 3. Mix Text and non-Text data model

### 3.1 Non-text Features +  tfidf features

In [124]:
final_data_text.head()

,targetSentiment,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,0,like animation animal look real,2.0,2.0,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,0,amazing realistic incredible music,3.0,0.0,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,0,classic good remake love glover outstanding,4.0,1.0,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,1,nice animation cgi completely lack disney styl...,5.0,2.0,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,0,good mainly sentimental value,2.0,1.0,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0


In [125]:
X = final_data_text.drop(targetSentiment,axis=1)
y = final_data_text[targetSentiment]

In [126]:
X.head()

,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,like animation animal look real,2.0,2.0,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,amazing realistic incredible music,3.0,0.0,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,classic good remake love glover outstanding,4.0,1.0,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,nice animation cgi completely lack disney styl...,5.0,2.0,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,good mainly sentimental value,2.0,1.0,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0


### train-test split

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [128]:
### tfidf matrix

# Taking both unigram and bigram
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, max_features=10000000,
                                 min_df=0.01,
                                 use_idf=True, ngram_range=(1,2))


tfidf_vectorizer.fit(X_train[clean_after_antonymns])

X_train_tfidf_matrix= tfidf_vectorizer.transform(X_train[clean_after_antonymns])
X_test_tfidf_matrix = tfidf_vectorizer.transform(X_test[clean_after_antonymns])


In [129]:
X_train_tfidf_matrix

<2400x222 sparse matrix of type '<class 'numpy.float64'>'
	with 16793 stored elements in Compressed Sparse Row format>

In [130]:
X_test_tfidf_matrix

<600x222 sparse matrix of type '<class 'numpy.float64'>'
	with 4464 stored elements in Compressed Sparse Row format>

In [131]:
print(tfidf_vectorizer.get_feature_names())

['absolutely', 'absolutely love', 'act', 'acting', 'action', 'actor', 'add', 'age', 'amazing', 'animal', 'animal look', 'animate', 'animate version', 'animation', 'apathy', 'away', 'awesome', 'bad', 'beautiful', 'beautifully', 'better', 'beyonce', 'big', 'bit', 'bore', 'boring', 'bring', 'cartoon', 'cartoon version', 'cast', 'cgi', 'change', 'character', 'child', 'childhood', 'cinematography', 'classic', 'close', 'come', 'compare', 'cry', 'cute', 'daughter', 'definitely', 'destroy', 'different', 'disappointed', 'dislike', 'disney', 'disney movie', 'earl', 'earl jone', 'effect', 'emotion', 'emotional', 'enjoy', 'enjoy movie', 'enjoyable', 'entertaining', 'especially', 'exactly', 'excellent', 'expect', 'experience', 'expression', 'face', 'facial', 'fact', 'fake', 'fall', 'family', 'fantastic', 'far', 'favorite', 'feel', 'feel like', 'film', 'flat', 'follow', 'forget', 'fun', 'funny', 'get', 'give', 'glad', 'go', 'good', 'good movie', 'graphic', 'great', 'great job', 'great movie', 'grow'

In [132]:
X_train_tfidf_matrix

<2400x222 sparse matrix of type '<class 'numpy.float64'>'
	with 16793 stored elements in Compressed Sparse Row format>

In [133]:
X_train_tfidf_df = pd.DataFrame(X_train_tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names(),index=X_train.index)
X_test_tfidf_df  = pd.DataFrame(X_test_tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names(),index=X_test.index)

In [134]:
X_train_tfidf_df.head()

,absolutely,absolutely love,act,acting,action,actor,add,age,amazing,animal,...,watch,way,well,wish,wonderful,work,worth,year,year old,young
642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.226768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1697,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.169833,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
X_train_remaining = X_train.drop([clean_after_antonymns],axis=1)
X_test_remaining = X_test.drop([clean_after_antonymns],axis=1)


In [136]:
X_train_remaining.head()

,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
642,5.0,2.0,0.393087,0.487273,0.00,22,0,2,1,0,1,0,0
700,8.0,2.0,0.427778,0.005952,0.25,39,0,0,0,0,0,1,0
226,1.0,1.0,0.875000,0.500000,0.50,4,0,1,0,0,1,0,0
1697,1.0,2.0,0.412500,0.400000,0.00,19,0,1,0,4,6,1,0
1010,11.0,8.0,0.230303,0.281481,0.00,82,0,1,2,0,0,1,0


In [137]:
if 1==0:
    X_train_remaining.reset_index(drop=True,inplace=True)
    X_test_remaining.reset_index(drop=True,inplace=True)

    y_train.reset_index(drop=True,inplace=True)
    y_test.reset_index(drop=True,inplace=True)



In [138]:
final_X_train = pd.concat([X_train_remaining,X_train_tfidf_df],axis=1)
final_X_test = pd.concat([X_test_remaining,X_test_tfidf_df],axis=1)

In [139]:
print("-------------------------------------")
print("final_X shape")
print(final_X_train.shape)
print(final_X_test.shape)

print("-------------------------------------")
print("X _remaining shape")
print(X_train_remaining.shape)
print(X_test_remaining.shape)

print("-------------------------------------")
print("X _tfidf shape")
print(X_train_tfidf_df.shape)
print(X_test_tfidf_df.shape)

-------------------------------------
final_X shape
(2400, 235)
(600, 235)
-------------------------------------
X _remaining shape
(2400, 13)
(600, 13)
-------------------------------------
X _tfidf shape
(2400, 222)
(600, 222)


#### SMOTE the train data as there is data imbalance

In [140]:
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_sample(final_X_train, y_train)

In [141]:
print(f"final_X_train.shape = {final_X_train.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_train_sm.shape = {X_train_sm.shape}")
print(f"y_train_sm.shape = {y_train_sm.shape}")
print(f"sum(y_train_sm) = {sum(y_train_sm)}")

final_X_train.shape = (2400, 235)
y_train.shape = (2400,)
X_train_sm.shape = (3468, 235)
y_train_sm.shape = (3468,)
sum(y_train_sm) = 1734


### 3.1.1 Logistic Regression

In [142]:
X_test=final_X_test

C=[0.001,0.01,0.1,1,10,100]
Penalty=['l1','l2']
        
logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name="Non-Text+TFIDF")

-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.0

Train data f1-score for class '1' 0.6666666666666666
TEST DATA ACCURACY 0.27

Test data f1-score for class '0' 0.0

Test data f1-score for class '1' 0.4251968503937008
-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l2
TRAIN DATA ACCURACY 0.7698961937716263

Train data f1-score for class '0' 0.7615062761506276

Train data f1-score for class '1' 0.777715877437326
TEST DATA ACCURACY 0.7816666666666666

Test data f1-score for class '0' 0.8376703841387856

Test data f1-score for class '1' 0.6666666666666666
-------------------------------------------------------------------------------------
C :  0.01 Penalty :  l1
TRAIN DATA ACCURACY 0.7647058823529411

Train data f1-score for class '0' 0.7588652482269505

Train data f1-score for class '1' 0.7702702702702703

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 3.1.2 XG Boost

In [143]:
X_test = final_X_test

param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),
    'n_estimators':[100],
    'max_depth': [10]
}

xgb_model(X_train_sm,y_train_sm,X_test,param_grid,"Non-Text+TFIDF")

-------------------------------------------------------------------------------------
CV_XGB.best_score_  0.8872549019607843
CV_XGB.best_params_  {'colsample_bytree': 0.8, 'max_depth': 10, 'n_estimators': 100}
TRAIN DATA ACCURACY 0.9769319492502884

Train data f1-score for class '0' 0.9769186381996539

Train data f1-score for class '1' 0.9769452449567724
TEST DATA ACCURACY 0.8383333333333334

Test data f1-score for class '0' 0.888633754305396

Test data f1-score for class '1' 0.7051671732522796


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 3.1.3 RandomForest

In [144]:
X_test = final_X_test

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20],
           "class_weight" : ['balanced','balanced_subsample']}


rfc_model(X_train_sm,y_train_sm,X_test,param_grid,"Non-Text+TFIDF") 

-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.8405420991926182
rfc_cv_grid.best_params_  {'n_estimators': 25, 'min_samples_leaf': 5, 'max_depth': 20, 'class_weight': 'balanced'}
TRAIN DATA ACCURACY 0.9048442906574394

Train data f1-score for class '0' 0.9028268551236749

Train data f1-score for class '1' 0.9067796610169492
TEST DATA ACCURACY 0.8233333333333334

Test data f1-score for class '0' 0.8741092636579573

Test data f1-score for class '1' 0.7039106145251396


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [10, 25, 50, 75, 100], 'max_depth': [10, 12, 14, 16, 18, 20], 'min_samples_leaf': [5, 10, 15, 20], 'class_weight': ['balanced', 'balanced_subsample']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

### 3.2 Non-text Features +  CountVectorizer features

In [145]:
final_data_text.head()

,targetSentiment,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
0,0,like animation animal look real,2.0,2.0,0.433333,0.200000,0.000000,12,0,0,0,0,0,0,0
1,0,amazing realistic incredible music,3.0,0.0,0.605556,0.555556,0.000000,6,0,2,0,0,1,0,0
2,0,classic good remake love glover outstanding,4.0,1.0,0.516667,0.466667,0.500000,8,0,2,1,0,0,0,0
3,1,nice animation cgi completely lack disney styl...,5.0,2.0,0.318750,0.391667,0.100000,27,1,0,1,1,0,0,0
4,0,good mainly sentimental value,2.0,1.0,0.205556,0.225000,0.166667,8,0,0,0,0,0,0,0


In [146]:
X = final_data_text.drop(targetSentiment,axis=1)
y = final_data_text[targetSentiment]

#### train-test split

In [147]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

#### CountVectorizer Matrix

In [148]:
# Taking both unigram and bigram
cvect = CountVectorizer(max_df=0.9, max_features=10000000,
                                 min_df=0.001, ngram_range=(1,2))

#cvect = CountVectorizer()
cvect.fit(X_train[clean_after_antonymns])

X_train_cvect_matrix= cvect.transform(X_train[clean_after_antonymns])
X_test_cvect_matrix = cvect.transform(X_test[clean_after_antonymns])

#### transform the FinalOut test data

In [149]:
FinalOut_cvect_matrix = cvect.transform(FinalOut_final_data_text[clean_after_antonymns])

#### Making CountVectorize matrix as DF

In [150]:
X_train_cvect_df = pd.DataFrame(X_train_cvect_matrix.toarray(), columns=cvect.get_feature_names(),index=X_train.index)
X_test_cvect_df  = pd.DataFrame(X_test_cvect_matrix.toarray(), columns=cvect.get_feature_names(),index=X_test.index)

In [151]:
X_train_cvect_df.head()

,able,absolute,absolutely,absolutely amazing,absolutely beautiful,absolutely love,absolutely stunning,accurate,achieve,achievement,...,year old,yes,young,young child,young kid,youth,yr,yr old,zazu,zimmer
642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
700,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
226,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Making FinalOut cvect matrix data to dataframe

In [152]:
FinalOut_cvect_df = pd.DataFrame(FinalOut_cvect_matrix.toarray(),columns=cvect.get_feature_names())

In [153]:
FinalOut_nontext_remaining = FinalOut_final_data_text.drop([clean_after_antonymns],axis=1)

In [154]:
X_train_remaining = X_train.drop([clean_after_antonymns],axis=1)
X_test_remaining = X_test.drop([clean_after_antonymns],axis=1)

In [155]:
X_train.head()

,antonymns_upd,adj_count,adv_verb_count,review_polar,adj_polar,adv_verb_polar,count_words,count_capital_words,count_love,count_shit,count_quest_marks,count_excl_marks,count_NOT_words,count_emojis
642,incredible visual effect bring warm feeling no...,5.0,2.0,0.393087,0.487273,0.00,22,0,2,1,0,1,0,0
700,good bad fake visual pretty big problem animal...,8.0,2.0,0.427778,0.005952,0.25,39,0,0,0,0,0,1,0
226,love movie,1.0,1.0,0.875000,0.500000,0.50,4,0,1,0,0,1,0,0
1697,lion king dislike music fabulous plan go,1.0,2.0,0.412500,0.400000,0.00,19,0,1,0,4,6,1,0
1010,bland colorless lack character song devoid ori...,11.0,8.0,0.230303,0.281481,0.00,82,0,1,2,0,0,1,0


In [156]:
final_X_train = pd.concat([X_train_remaining,X_train_cvect_df],axis=1)
final_X_test = pd.concat([X_test_remaining,X_test_cvect_df],axis=1)

#### concat FinalOut data for non text and count vector matrix like train data above

In [157]:
final_FinalOut_cvet_ntext = pd.concat([FinalOut_nontext_remaining,FinalOut_cvect_df],axis=1)

In [158]:
final_FinalOut_cvet_ntext.shape

(1200, 2304)

In [159]:
print("-------------------------------------")
print("final_X shape")
print(final_X_train.shape)
print(final_X_test.shape)

print("-------------------------------------")
print("X _remaining shape")
print(X_train_remaining.shape)
print(X_test_remaining.shape)

print("-------------------------------------")
print("X _tfidf shape")
print(X_train_cvect_df.shape)
print(X_test_cvect_df.shape)

-------------------------------------
final_X shape
(2400, 2304)
(600, 2304)
-------------------------------------
X _remaining shape
(2400, 13)
(600, 13)
-------------------------------------
X _tfidf shape
(2400, 2291)
(600, 2291)


In [160]:
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_sample(final_X_train, y_train)

In [161]:
print(f"final_X_train.shape = {final_X_train.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"X_train_sm.shape = {X_train_sm.shape}")
print(f"y_train_sm.shape = {y_train_sm.shape}")
print(f"sum(y_train_sm) = {sum(y_train_sm)}")

final_X_train.shape = (2400, 2304)
y_train.shape = (2400,)
X_train_sm.shape = (3468, 2304)
y_train_sm.shape = (3468,)
sum(y_train_sm) = 1734


### 3.2.1 Logistic Regression

In [162]:
X_test=final_X_test

C=[0.001,0.01,0.1,1,10,100]
Penalty=['l1','l2']
        
logistic_regr(X_train_sm,y_train_sm,X_test,C_list,Penalty,Features_name="Non-Text+CountVect")

-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l1
TRAIN DATA ACCURACY 0.5

Train data f1-score for class '0' 0.0

Train data f1-score for class '1' 0.6666666666666666
TEST DATA ACCURACY 0.27

Test data f1-score for class '0' 0.0

Test data f1-score for class '1' 0.4251968503937008
-------------------------------------------------------------------------------------
C :  0.001 Penalty :  l2
TRAIN DATA ACCURACY 0.7946943483275664

Train data f1-score for class '0' 0.7870813397129187

Train data f1-score for class '1' 0.8017817371937639
TEST DATA ACCURACY 0.8

Test data f1-score for class '0' 0.8536585365853657

Test data f1-score for class '1' 0.6842105263157895
-------------------------------------------------------------------------------------
C :  0.01 Penalty :  l1
TRAIN DATA ACCURACY 0.7724913494809689

Train data f1-score for class '0' 0.7663606751554634

Train data f1-score for class '1' 0.7783085136274235
TEST DATA ACC

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

### 3.2.2 XG Boost

In [163]:
X_test = final_X_test

param_grid = {
    'colsample_bytree': np.linspace(0.5, 0.9, 5),
    'n_estimators':[100],
    'max_depth': [10]
}

xgb_model(X_train_sm,y_train_sm,X_test,param_grid,"Non-Text+CountVect")

C:\Users\trex\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


-------------------------------------------------------------------------------------
CV_XGB.best_score_  0.8898500576701269
CV_XGB.best_params_  {'colsample_bytree': 0.7, 'max_depth': 10, 'n_estimators': 100}
TRAIN DATA ACCURACY 0.9760668973471741

Train data f1-score for class '0' 0.9762517882689556

Train data f1-score for class '1' 0.975879104911363
TEST DATA ACCURACY 0.8483333333333334

Test data f1-score for class '0' 0.8967082860385925

Test data f1-score for class '1' 0.7147335423197492


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 3.2.3 RandomForest

In [164]:
X_test = final_X_test

param_grid = {"n_estimators" : [10, 25, 50, 75, 100],
           "max_depth" : [10, 12, 14, 16, 18, 20],
           "min_samples_leaf" : [5, 10, 15, 20],
           "class_weight" : ['balanced','balanced_subsample']}

print(X_test.shape)

rfc_cv_grid=rfc_model(X_train_sm,y_train_sm,X_test,param_grid,"Non-Text+CountVect")

print(X_test.shape)

(600, 2304)
-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.845444059976932
rfc_cv_grid.best_params_  {'n_estimators': 50, 'min_samples_leaf': 5, 'max_depth': 20, 'class_weight': 'balanced_subsample'}
TRAIN DATA ACCURACY 0.8869665513264129

Train data f1-score for class '0' 0.8839550029603316

Train data f1-score for class '1' 0.8898257448004497
TEST DATA ACCURACY 0.835

Test data f1-score for class '0' 0.8820023837902263

Test data f1-score for class '1' 0.7257617728531857
(600, 2304)


In [167]:
print(X_test.shape)

(600, 2304)


## After different scenarios of ML model have run, analyse the model summary for the models giving best score

In [168]:
pd.DataFrame(model_summary).to_csv("Model_Summary_v1.csv")
print(print_line)
print(model_summary.head())
print(print_line)
print(model_summary.tail())

----------------------------------------------------------------------------------------------------------------------------
  Model_Name Train_Accuracy F1_0Class F1_1Class Test_Accuracy F1_0Class_test  \
0   Logistic            0.5         0  0.666667          0.27              0   
1   Logistic       0.767013  0.758084  0.775306         0.785       0.840149   
2   Logistic       0.761534  0.756263  0.766582      0.773333       0.831266   
3   Logistic       0.769608  0.765483   0.77359      0.791667       0.845105   
4   Logistic       0.775375  0.773217  0.777492      0.803333       0.853598   

  F1_1Class_test  Features                  Comments  
0       0.425197  Non-Text   C : 0.001| Penalty : l1  
1       0.671756  Non-Text   C : 0.001| Penalty : l2  
2       0.654822  Non-Text    C : 0.01| Penalty : l1  
3       0.681934  Non-Text    C : 0.01| Penalty : l2  
4       0.700508  Non-Text     C : 0.1| Penalty : l1  
----------------------------------------------------------------

## After analyzing the xls file, using the Logistic and RFC that is the best for final prediction

#### This model is based on 80:20 split of train data

#### Selecting the best 3 models : 2 logistc , 1 RFC

In [169]:
logisticRegr = LogisticRegression(penalty = 'l1', C = 1.0,random_state = 0)
logisticRegr.fit(X_train_sm, y_train_sm)

## Predicting for the given TEST data
lgr_FinalOut_ver1 = logisticRegr.predict(final_FinalOut_cvet_ntext)    
pd.DataFrame(lgr_FinalOut_ver1).to_csv("lgr_cvet_ntext_verl1_1.csv") ## Result .70 F1 score

In [170]:
logisticRegr = LogisticRegression(penalty = 'l2', C = 1.0,random_state = 0)
logisticRegr.fit(X_train_sm, y_train_sm)

## Predicting for the given TEST data
lgr_FinalOut_ver1_1 = logisticRegr.predict(final_FinalOut_cvet_ntext) 
pd.DataFrame(lgr_FinalOut_ver1_1).to_csv("lgr_cvet_ntext_verl2_1.csv") ## Result .69 F1 score

In [171]:
## Predicting for the given TEST data
rfc_FinalOut_ver1 = rfc_cv_grid.predict(final_FinalOut_cvet_ntext)

pd.DataFrame(rfc_FinalOut_ver1).to_csv("rfc_cvet_ntext_ver1.csv") ## Result : .71 F1 score

### Training on full data now

In [172]:
## Using the best model paramaters and training on full data

full_X_train=np.concatenate([X_train_sm,X_test.values])
full_y_train=np.concatenate([y_train_sm,y_test])
#X_test=final_FinalOut_cvet_ntext

print(X_train_sm.shape)
print(X_test.shape)
print(full_X_train.shape)
print(full_y_train.shape)

(3468, 2304)
(600, 2304)
(4068, 2304)
(4068,)


In [173]:
X_test = final_X_test
full_X_train=np.concatenate([X_train_sm,X_test.values])
full_y_train=np.concatenate([y_train_sm,y_test])
X_test=final_FinalOut_cvet_ntext

print(X_train_sm.shape)
print(X_test.shape)
print(full_X_train.shape)
print(full_y_train.shape)
## Using Best models for Predictions
## 1st :  logistic model
C_list=[1.0]
Penalty=['l1']
lgr=logistic_regr(full_X_train, full_y_train,X_test,C_list,Penalty,Features_name="FULL DATA: Non-Text+CountVect",TESTDATA=False)
    
lgr_FinalOut_ver_full2_1 = lgr.predict(final_FinalOut_cvet_ntext) ## will use this in stacking model
pd.DataFrame(lgr_FinalOut_ver_full2_1).to_csv("lgr_cvet_ntext_ver_full2_l1.csv") ## Result .72 F1 score


lgr_full_train_v2_1_train= lgr.predict(full_X_train) ## will use this pred values for generating stacking model

## 2nd :  logistic model

C_list=[1.0]
Penalty=['l2']
lgr=logistic_regr(full_X_train, full_y_train,X_test,C_list,Penalty,Features_name="FULL DATA: Non-Text+CountVect",TESTDATA=False)

lgr_FinalOut_ver_full2_2 = lgr.predict(final_FinalOut_cvet_ntext) ## will use this in stacking model

pd.DataFrame(lgr_FinalOut_ver_full2_2).to_csv("lgr_cvet_ntext_ver_full2_l2.csv")  ## Result .70 F1 score

lgr_full_train_v2_2_train = lgr.predict(full_X_train) ## will use this pred values for generating stacking model

## 3rd :  Random Forest model

# using the best paramaters
param_grid = {"n_estimators" : [50],
           "max_depth" : [20],
           "min_samples_leaf" : [5],
           "class_weight" : ['balanced']}


rfc_cv_grid=rfc_model(full_X_train, full_y_train,X_test,param_grid,"FULL DATA : Non-Text+CountVect",TESTDATA=False)

rfc_FinalOut_ver_full2 = rfc_cv_grid.predict(final_FinalOut_cvet_ntext)
pd.DataFrame(rfc_FinalOut_ver_full2).to_csv("rfc_cvet_ntext_ver2_.csv") ## Result : .69 F1 score


rfc_full_2_train = rfc_cv_grid.predict(full_X_train) ## will use this pred values for generating stacking model

(3468, 2304)
(1200, 2304)
(4068, 2304)
(4068,)
-------------------------------------------------------------------------------------
C :  1.0 Penalty :  l1
TRAIN DATA ACCURACY 0.9456735496558505

Train data f1-score for class '0' 0.9486404833836858

Train data f1-score for class '1' 0.9423428124184712
-------------------------------------------------------------------------------------
C :  1.0 Penalty :  l2
TRAIN DATA ACCURACY 0.9650934119960669

Train data f1-score for class '0' 0.9670991658943465

Train data f1-score for class '1' 0.9628272251308899
-------------------------------------------------------------------------------------
rfc_cv_grid.best_score_  0.8470993117010817
rfc_cv_grid.best_params_  {'n_estimators': 50, 'min_samples_leaf': 5, 'max_depth': 20, 'class_weight': 'balanced'}
TRAIN DATA ACCURACY 0.8842182890855457

Train data f1-score for class '0' 0.8885207100591717

Train data f1-score for class '1' 0.8795704423421121


### Analysis of Model Summary

### Stacking on Full data

In [174]:
stack_train = pd.DataFrame([rfc_full_2_train,lgr_full_train_v2_1_train,lgr_full_train_v2_2_train])

In [175]:
stack_train = stack_train.T

In [176]:
stack_train.head()

,0,1,2
0,0,0,0
1,1,0,0
2,0,0,0
3,0,0,0
4,1,1,1


In [177]:
stack_train = stack_train.as_matrix()

In [178]:
stack_train.shape

(4068, 3)

In [179]:
full_y_train.shape

(4068,)

In [180]:
param_grid = {
     'colsample_bytree': np.linspace(0.2, 0.9, 10),
     'n_estimators':[100],
     'max_depth': [10]
}

XGB = XGBClassifier(n_jobs=-1)
stack_XGB = GridSearchCV(estimator=XGB, param_grid=param_grid, cv= 10)

In [181]:
stack_XGB.fit(stack_train,full_y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'colsample_bytree': array([0.2    , 0.27778, 0.35556, 0.43333, 0.51111, 0.58889, 0.66667,
       0.74444, 0.82222, 0.9    ]), 'n_estimators': [100], 'max_depth': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [182]:
stack_XGB.best_score_

0.9650934119960669

In [183]:
FinalOut_stack_train = pd.DataFrame([rfc_FinalOut_ver_full2,lgr_FinalOut_ver_full2_1,lgr_FinalOut_ver_full2_2])

In [184]:
FinalOut_stack_train = FinalOut_stack_train.T

FinalOut_stack_train = FinalOut_stack_train.as_matrix()

In [185]:
FinalOut_stack_train.shape

(1200, 3)

In [186]:
stack_predict_FinalOut =  stack_XGB.predict(FinalOut_stack_train)

In [187]:
pd.DataFrame(stack_predict_FinalOut).to_csv("stack_pred_FinalOut.csv") ## Result  .71 F1 score

<div class="alert alert-block alert-warning" align=center>
</div>